In [1]:
from arhender_aiida_workchains.workchains import DiffuseImplicitHybridWorkChain
from aiida.orm import Str, Dict, load_code
from aiida.engine import submit
from aiida import load_profile

load_profile()

builder = DiffuseImplicitHybridWorkChain.get_builder()

with open('basis_file.txt','r') as bf:
    diffuse_basis_set_data = bf.readlines()

df = ''
for line in diffuse_basis_set_data:
    df += line
df = Str(df)


g16_base_params = Dict({'charge': -1,
    'basis_set': '6-31g(d)',
    'functional': 'hf',
    'multiplicity': 1,
    'input_parameters': {},
    'link0_parameters': {'%chk': 'aiida.chk',
    '%mem': '1600MB',
    '%nprocshared': 4},
    'route_parameters': {'opt': None, 'freq': None}})


with open('structure.txt','r') as sf:
    structure = sf.readlines()
structr=''
for line in structure:
    structr += line
structr = Str(structr)

molecule_name = Str('PbiL2mL1_Conformers_13')
solvent=  Str('water')
builder.input_structure = structr
builder.solvent = solvent
builder.molecule_name = molecule_name
builder.diffuse_basis_set_data = df
builder.g16_code = load_code('gaussian@cedar')
builder.original_parsed_data = Dict({'freeenergy':123})
builder.g16_base_params = g16_base_params
node = submit(builder)